In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import tensorflow as tf
from helper_functions import *
import pandas as pd
# importTensorflow(memory=4090)

In [2]:
df = pd.read_csv("queries1.csv")
df.head()

,String,Class
0,Duck or goose liver recipes,foie_gras
1,French delicacy made from liver,foie_gras
2,Luxury food items from France,foie_gras
3,How to cook liver in a luxurious way,foie_gras
4,Dishes using corn-fed duck liver,foie_gras


In [19]:
class_list = os.listdir("test/")

In [3]:
d = set()
counter = -1
l = []
for thing in df.Class:
    if thing in d:
      l.append(counter)
    else:
        counter += 1
        d.add(thing)
        l.append(counter)

In [5]:
df['classEncode'] = l

In [6]:
df.head()

,String,Class,classEncode
0,Duck or goose liver recipes,foie_gras,0
1,French delicacy made from liver,foie_gras,0
2,Luxury food items from France,foie_gras,0
3,How to cook liver in a luxurious way,foie_gras,0
4,Dishes using corn-fed duck liver,foie_gras,0


In [7]:
df = df.sample(frac = 1)
df.head()

,String,Class,classEncode
3896,Recipes using dough and tomatoes,pizza,79
3933,How to make a dish with ground beef and cheese,nachos,80
2795,Recipe for seafood with butter and white wine,scallops,57
352,Making a spicy snack with a dough wrapper,samosa,7
1277,How to prepare a dish with spaghetti and a me...,spaghetti_bolognese,25


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.String, df.classEncode, test_size=0.1, random_state=69)

In [9]:
len(X_train), len(X_test)

(4488, 499)

In [10]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64).prefetch(tf.data.AUTOTUNE)
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64).prefetch(tf.data.AUTOTUNE)

In [11]:
vocab_length = 25000
max_length = [len(i.split()) for i in df.String]
max_length = int(np.percentile(max_length, 95))
max_length

12

In [12]:
#vectorization
text_vector = tf.keras.layers.TextVectorization(max_tokens=vocab_length,
                                                output_sequence_length=max_length)

In [13]:
text_vector.adapt(df.String)

In [14]:
text_vector.get_vocabulary()[:5]

['', '[UNK]', 'with', 'a', 'and']

In [15]:
# embedding
embedding = tf.keras.layers.Embedding(input_dim = vocab_length,
                                      input_length=max_length,
                                      output_dim=1024,
                                      mask_zero=True)

In [16]:
one_hot_encoded_train = tf.one_hot(y_train, depth=101)
one_hot_encoded_test = tf.one_hot(y_test, depth=101)

In [17]:
train_data_one_hot = tf.data.Dataset.from_tensor_slices((X_train, one_hot_encoded_train)).shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)
test_data_one_hot = tf.data.Dataset.from_tensor_slices((X_test, one_hot_encoded_test)).batch(64).prefetch(tf.data.AUTOTUNE)

In [20]:
# conv1d
inputs = tf.keras.layers.Input(shape=(1, ), dtype='string')
x = text_vector(inputs)
x = embedding(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True))(x)
x = tf.keras.layers.GRU(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(class_list), activation='softmax')(x)

# lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch : 3e-3/(1+epoch)**(2))

model = tf.keras.models.Model(inputs, outputs)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 12)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 12, 1024)          25600000  
                                                                 
 bidirectional_1 (Bidirecti  (None, 12, 1024)          4724736   
 onal)                                                           
                                                                 
 gru_3 (GRU)                 (None, 128)               443136    
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512   

In [21]:
history = model.fit(train_data_one_hot,
                    epochs=5,
                    validation_data=(test_data_one_hot),
                    validation_steps=int(0.25*len(test_data_one_hot)),
                    steps_per_epoch=len(train_data_one_hot))

Epoch 1/5


71/71 [==============================] - 30s 352ms/step - loss: 2.4011 - accuracy: 0.4514 - val_loss: 1.2423 - val_accuracy: 0.7969
Epoch 2/5
71/71 [==============================] - 23s 321ms/step - loss: 0.4398 - accuracy: 0.8734 - val_loss: 0.3751 - val_accuracy: 0.9297
Epoch 3/5
71/71 [==============================] - 23s 328ms/step - loss: 0.3234 - accuracy: 0.9000 - val_loss: 0.3826 - val_accuracy: 0.8984
Epoch 4/5
71/71 [==============================] - 24s 340ms/step - loss: 0.2320 - accuracy: 0.9245 - val_loss: 0.3000 - val_accuracy: 0.9219
Epoch 5/5
71/71 [==============================] - 25s 349ms/step - loss: 0.2158 - accuracy: 0.9356 - val_loss: 0.2610 - val_accuracy: 0.9219


In [22]:
model.evaluate(test_data_one_hot)

8/8 [==============================] - 1s 129ms/step - loss: 0.3576 - accuracy: 0.8958


[0.3576110601425171, 0.8957915902137756]

In [42]:
model.save("bidir_gru/")

INFO:tensorflow:Assets written to: bidir_gru/assets


INFO:tensorflow:Assets written to: bidir_gru/assets


In [23]:
tf.argmax(model.predict(["recipes with Mustard and bread"]),axis=1)

1/1 [==============================] - 2s 2s/step


<tf.Tensor: shape=(1,), dtype=int64, numpy=array([75])>

In [24]:
class_list[75]

'tacos'

In [31]:
class_list[tf.argmax(model.predict(["Dinner recipes with beef and jalapenos"]), axis=1).numpy()[0]]

1/1 [==============================] - 0s 64ms/step


'tacos'

In [33]:
class_list[30]

'bruschetta'

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('queries1.csv')

X = df['String']
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline that first transforms your data using TF-IDF and then fits it to a Naive Bayes classifier
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train your model using the training data
model.fit(X_train, y_train)

# Evaluate your model using the testing data
print("Model Accuracy: ", model.score(X_test, y_test))

Model Accuracy:  0.8837675350701403


In [47]:
# Your custom string
custom_string = "sausage and bread"

# Transform your string into a TF-IDF vector
custom_string_transformed = model.named_steps['tfidfvectorizer'].transform([custom_string])

# Make prediction
prediction = model.named_steps['multinomialnb'].predict(custom_string_transformed)

print(prediction)

['hot_dog']


In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import pandas as pd

# Load your data
df = pd.read_csv('queries1.csv')

X = df['String']
y = df['Class']

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline that first transforms your data using TF-IDF and then fits it to a SVC
model = make_pipeline(TfidfVectorizer(), SVC(probability=True))

# Train your model using the training data
model.fit(X_train, y_train)

# Evaluate your model using the testing data
print("Model Accuracy: ", model.score(X_test, y_test))

Model Accuracy:  0.9198396793587175


In [57]:
# Your custom string
custom_string = "different cheese"

# Transform your string into a TF-IDF vector
custom_string_transformed = model.named_steps['tfidfvectorizer'].transform([custom_string])

# Make prediction
prediction = model.named_steps['svc'].predict(custom_string_transformed)

print(prediction)

['cheese_plate']


In [77]:
from joblib import dump

# Save the model to a file
dump(model, 'model.joblib')

['model.joblib']

In [59]:
from joblib import load

# Load the model from a file
model_2 = load('model.joblib')

In [60]:
# Your custom string
custom_string = "different cheese"

# Transform your string into a TF-IDF vector
custom_string_transformed = model_1.named_steps['tfidfvectorizer'].transform([custom_string])

# Make prediction
prediction = model_1.named_steps['svc'].predict(custom_string_transformed)

print(prediction)

['cheese_plate']


In [75]:
# Your custom string
custom_string = "dough cheese and tomato"

# Transform your string into a TF-IDF vector
custom_string_transformed = model.named_steps['tfidfvectorizer'].transform([custom_string])

# Get probabilities
probabilities = model.named_steps['svc'].predict_proba(custom_string_transformed)

# Get the indices of the top 5 predictions
top5_indices = probabilities[0].argsort()[-5:][::-1]

# Get the class labels
class_labels = model.named_steps['svc'].classes_

# Print the top 5 predicted dishes
for i in top5_indices:
    print(class_labels[i])

pizza
dumplings
cheese_plate
lasagna
foie_gras


In [76]:
sorted(probabilities[0],reverse=True)[:5],sum(sorted(probabilities[0],reverse=True))

([0.4377584584280028,
  0.04270574203028125,
  0.02681058380264024,
  0.019028585013695108,
  0.013631286282075097],
 0.9999999999999994)

In [78]:
from joblib import load

# Load the model from a file
model_2 = load('model.joblib')

In [88]:
# Your custom string
custom_string = "ham and cheese"

# Transform your string into a TF-IDF vector
custom_string_transformed = model.named_steps['tfidfvectorizer'].transform([custom_string])

# Get probabilities
probabilities = model.named_steps['svc'].predict_proba(custom_string_transformed)

# Get the indices of the top 5 predictions
top5_indices = probabilities[0].argsort()[-5:][::-1]

# Get the class labels
class_labels = model.named_steps['svc'].classes_

# Print the top 5 predicted dishes
for i in top5_indices:
    print(class_labels[i])

croque_madame
cheese_plate
pizza
foie_gras
nachos


In [89]:
probs = sorted(probabilities[0],reverse=True)[:5]
for i,idx in enumerate(top5_indices):
    print(class_labels[idx],probs[i])


croque_madame 0.17546075010608803
cheese_plate 0.05577163843944749
pizza 0.02578475714521297
foie_gras 0.02558948991359066
nachos 0.017885596906728963


In [27]:
import os

class_list = os.listdir("test/")

In [3]:
class_list

['foie_gras',
 'club_sandwich',
 'cheese_plate',
 'cup_cakes',
 'garlic_bread',
 'gnocchi',
 'ice_cream',
 'samosa',
 'donuts',
 'tuna_tartare',
 'filet_mignon',
 'seaweed_salad',
 'french_toast',
 'chicken_curry',
 'shrimp_and_grits',
 'steak',
 'cheesecake',
 'red_velvet_cake',
 'waffles',
 'churros',
 'gyoza',
 'lobster_roll_sandwich',
 'huevos_rancheros',
 'breakfast_burrito',
 'grilled_cheese_sandwich',
 'spaghetti_bolognese',
 'falafel',
 'poutine',
 'greek_salad',
 'beef_tartare',
 'fried_calamari',
 'guacamole',
 'ravioli',
 'lobster_bisque',
 'beet_salad',
 'risotto',
 'crab_cakes',
 'strawberry_shortcake',
 'edamame',
 'ceviche',
 'hot_and_sour_soup',
 'spring_rolls',
 'sashimi',
 'paella',
 'clam_chowder',
 'miso_soup',
 'escargots',
 'hot_dog',
 'pulled_pork_sandwich',
 'bruschetta',
 'panna_cotta',
 'fish_and_chips',
 'pad_thai',
 'tiramisu',
 'takoyaki',
 'macarons',
 'apple_pie',
 'cannoli',
 'scallops',
 'frozen_yogurt',
 'chicken_quesadilla',
 'mussels',
 'beef_carpacc

In [4]:
import pandas as pd
import yaml

with open('recipes.yaml', 'r') as file:
    data = yaml.safe_load(file)

df = pd.json_normalize(data)

In [6]:
print(df.head())

                                             recipes
0  [{'name': 'Foie Gras', 'ingredients': ['Fresh ...


In [11]:
import pandas as pd
import yaml

with open('recipes.yaml', 'r') as file:
    data = yaml.safe_load(file)

# Convert the entire data list into a DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df.head())

                                             recipes
0  {'name': 'Foie Gras', 'ingredients': ['Fresh d...
1  {'name': 'Club Sandwich', 'ingredients': ['Sli...
2  {'name': 'Gnocchi', 'ingredients': ['Potatoes'...
3  {'name': 'Ice Cream', 'ingredients': ['Heavy c...
4  {'name': 'Samosa', 'ingredients': ['Potatoes',...


In [13]:
# Apply pd.Series to each row to split the dictionary keys into separate columns
df_new = df['recipes'].apply(pd.Series)

# Print the new DataFrame
df_new.head()

,name,ingredients,steps
0,Foie Gras,"[Fresh duck/goose liver, Salt, White pepper, S...","[Remove veins from liver., Season with salt, p..."
1,Club Sandwich,"[Sliced bread, Turkey/Chicken breast, Bacon, L...","[Toast bread., Layer with turkey/chicken, baco..."
2,Gnocchi,"[Potatoes, Flour, Egg, Salt]","[Cook potatoes until tender., Mash and mix wit..."
3,Ice Cream,"[Heavy cream, Whole milk, Sugar, Vanilla extra...","[Combine ingredients., Freeze in ice cream mak..."
4,Samosa,"[Potatoes, Peas, Spices (such as cumin, corian...","[Boil potatoes., Sauté onion, garlic, ginger, ..."


In [15]:
import pandas as pd

In [22]:
df_new.to_csv("recipes.csv",index=False)

In [25]:
df1 = pd.read_csv("recipes.csv")
df1.head()

,name,ingredients,steps
0,Foie Gras,"['Fresh duck/goose liver', 'Salt', 'White pepp...","['Remove veins from liver.', 'Season with salt..."
1,Club Sandwich,"['Sliced bread', 'Turkey/Chicken breast', 'Bac...","['Toast bread.', 'Layer with turkey/chicken, b..."
2,Gnocchi,"['Potatoes', 'Flour', 'Egg', 'Salt']","['Cook potatoes until tender.', 'Mash and mix ..."
3,Ice Cream,"['Heavy cream', 'Whole milk', 'Sugar', 'Vanill...","['Combine ingredients.', ""Freeze in ice cream ..."
4,Samosa,"['Potatoes', 'Peas', 'Spices (such as cumin, c...","['Boil potatoes.', 'Sauté onion, garlic, ginge..."


In [31]:
df1['name']

0         Foie Gras
1     Club Sandwich
2           Gnocchi
3         Ice Cream
4            Samosa
          ...      
83          Baklava
84     Crème Brûlée
85      Carrot Cake
86      Onion Rings
87     Caesar Salad
Name: name, Length: 88, dtype: object

In [28]:
# # Read the dishes from the CSV file
# df1 = pd.read_csv("recipes.csv")

# Create a dictionary mapping dish names to classes
dish_to_class = dict(zip(df1['name'], class_list))

# Now you can access the class of a dish like this:
print(dish_to_class['Foie Gras'])

foie_gras


In [30]:
class_list

['foie_gras',
 'club_sandwich',
 'cheese_plate',
 'cup_cakes',
 'garlic_bread',
 'gnocchi',
 'ice_cream',
 'samosa',
 'donuts',
 'tuna_tartare',
 'filet_mignon',
 'seaweed_salad',
 'french_toast',
 'chicken_curry',
 'shrimp_and_grits',
 'steak',
 'cheesecake',
 'red_velvet_cake',
 'waffles',
 'churros',
 'gyoza',
 'lobster_roll_sandwich',
 'huevos_rancheros',
 'breakfast_burrito',
 'grilled_cheese_sandwich',
 'spaghetti_bolognese',
 'falafel',
 'poutine',
 'greek_salad',
 'beef_tartare',
 'fried_calamari',
 'guacamole',
 'ravioli',
 'lobster_bisque',
 'beet_salad',
 'risotto',
 'crab_cakes',
 'strawberry_shortcake',
 'edamame',
 'ceviche',
 'hot_and_sour_soup',
 'spring_rolls',
 'sashimi',
 'paella',
 'clam_chowder',
 'miso_soup',
 'escargots',
 'hot_dog',
 'pulled_pork_sandwich',
 'bruschetta',
 'panna_cotta',
 'fish_and_chips',
 'pad_thai',
 'tiramisu',
 'takoyaki',
 'macarons',
 'apple_pie',
 'cannoli',
 'scallops',
 'frozen_yogurt',
 'chicken_quesadilla',
 'mussels',
 'beef_carpacc

In [29]:
dish_to_class

{'Foie Gras': 'foie_gras',
 'Club Sandwich': 'club_sandwich',
 'Gnocchi': 'cheese_plate',
 'Ice Cream': 'cup_cakes',
 'Samosa': 'garlic_bread',
 'Donuts': 'gnocchi',
 'Tuna Tartare': 'ice_cream',
 'Filet Mignon': 'samosa',
 'Seaweed Salad': 'donuts',
 'French Toast': 'tuna_tartare',
 'Chicken Curry': 'filet_mignon',
 'Shrimp and Grits': 'seaweed_salad',
 'Steak': 'french_toast',
 'Cheesecake': 'chicken_curry',
 'Red Velvet Cake': 'shrimp_and_grits',
 'Waffles': 'steak',
 'Churros': 'cheesecake',
 'Gyoza': 'red_velvet_cake',
 'Lobster Roll Sandwich': 'waffles',
 'Huevos Rancheros': 'churros',
 'Breakfast Burrito': 'gyoza',
 'Grilled Cheese Sandwich': 'lobster_roll_sandwich',
 'Spaghetti Bolognese': 'huevos_rancheros',
 'Falafel': 'breakfast_burrito',
 'Poutine': 'grilled_cheese_sandwich',
 'Greek Salad': 'spaghetti_bolognese',
 'Beef Tartare': 'falafel',
 'Fried Calamari': 'poutine',
 'Guacamole': 'greek_salad',
 'Ravioli': 'beef_tartare',
 'Lobster Bisque': 'fried_calamari',
 'Beet Sal

In [33]:
len(df1['name'])

88

In [36]:
import yaml
import pandas as pd

with open('recipes1.yaml', 'r') as file:
    data = yaml.safe_load(file)

df = pd.json_normalize(data)

df

,foie_gras.ingredients,foie_gras.recipe,club_sandwich.ingredients,club_sandwich.recipe,cheese_plate.ingredients,cheese_plate.recipe,cup_cakes.ingredients,cup_cakes.recipe,garlic_bread.ingredients,garlic_bread.recipe,gnocchi.ingredients,gnocchi.recipe,ice_cream.ingredients,ice_cream.recipe,samosa.ingredients,samosa.recipe,donuts.ingredients,donuts.recipe,tuna_tartare.ingredients,tuna_tartare.recipe
0,"[1 whole fresh duck or goose liver, Salt, Pepp...",1. Remove any veins or membranes from the live...,"[Sliced bread, Turkey or chicken breast, Bacon...",1. Toast three slices of bread. 2. Cook bacon ...,"[Assorted cheeses (such as brie, cheddar, goud...",1. Arrange the assorted cheeses on a serving p...,"[All-purpose flour, Sugar, Baking powder, Salt...",1. Preheat oven to 350°F (175°C). Line a muffi...,"[French bread or baguette, Butter, Garlic clov...",1. Preheat oven to 375°F (190°C). 2. Slice the...,"[Potatoes, Flour, Egg, Salt]","1. Boil potatoes until tender, then peel and m...","[Heavy cream, Whole milk, Sugar, Vanilla extract]","1. In a bowl, whisk together heavy cream, whol...","[Potatoes, Peas, Carrots, Onion, Garlic, Ginge...","1. Boil potatoes until tender, then peel and m...","[All-purpose flour, Sugar, Yeast, Milk, Egg, B...","1. In a bowl, combine flour, sugar, yeast, and...","[Sushi-grade tuna, Soy sauce, Sesame oil, Lime...",1. Dice sushi-grade tuna into small cubes. 2. ...
